In [13]:
#To make predictions on the test set I will use a gradient boosting method.
#For more info see: https://en.wikipedia.org/wiki/Gradient_boosting

import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

train_loc = r"C:\Users\yashwant\Desktop\axis\dataset\train.csv"
test_loc = r"C:\Users\yashwant\Desktop\axis\dataset\test.csv"
submit_loc = r"C:\Users\yashwant\Desktop\axis\dataset\submit.csv"

train = pd.read_csv(train_loc)
test = pd.read_csv(test_loc)

In [14]:
#Create dummy variables then shuffle the train set.

cols = ['var_d1', 'var_d2', 'var_d3', 'var_d4', 'var_d5', 'var_d6', 'var_d7', 'var_d8', 'var_d9', 'var_d11']

train = pd.get_dummies(train, columns = cols)
test = pd.get_dummies(test, columns = cols)

train = train.sample(frac=1).reset_index(drop=True)

In [15]:
y = train.target


In [16]:
#Drop the unnecessary columns and prepare or data for our submission file.

train =  train.drop(['APP_ID_C', 'target'], axis=1)
X = train.astype(np.float32)
submit = test['APP_ID_C']


In [17]:
test = test.drop('APP_ID_C', axis=1)

In [18]:
#Use Scikit-Learn's train_test_split function to create train and validation sets.

X_train, X_val, y_train, yield_val = train_test_split(X, y, train_size=0.8)


In [19]:
#Create a function to run a gradient boosted classifier over our data. The reason we have to
#do this is due to the way Python uses parallelization on Windows.
#https://www.kaggle.com/c/malware-classification/forums/t/12802/
#anyone-getting-parallelizing-error-for-scikit-learn-based-models-in-python/66187#post66187

#Note numerous different values were used in the param_grid to hone in on the best paramater
#combinations. The param grid below is the final one I used.

def model(X_train, X_val, y_train, y_val):
    if __name__ == '__main__':
    
        param_grid = {'learning_rate': [0.03, 0.035],
                      'max_depth': [3, 4, 5],
                      'min_samples_leaf': [17, 18],
                      'max_features': [1.0, 0.95, 0.9],
                      'n_estimators': [100, 300, 500]
                      }

        estimator = GridSearchCV(estimator=GradientBoostingClassifier(),
                                 param_grid=param_grid,
                                 n_jobs=-1)

        estimator.fit(X_train, y_train)

        best_params = estimator.best_params_
                                 
        validation_accuracy = estimator.score(X_val, y_val)
        print('Validation accuracy: ', validation_accuracy)
        
        return best_params

In [20]:
#params that appeared most often after running the model ten times.
    
params = {'min_samples_leaf': 17, 'max_features': 0.95, 'max_depth': 3,
          'learning_rate': 0.03, 'n_estimators': 500}

In [21]:
#Fit model using our data and the best parameters found by GridSearchCV.

model = GradientBoostingClassifier(**params)
model.fit(X, y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.03, loss='deviance', max_depth=3,
              max_features=0.95, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=17, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=500,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [22]:
#Make predictions on the test set.

preds = model.predict(test)

ValueError: Number of features of the model must match the input. Model n_features is 697 and input n_features is 658 